In [ ]:
source(paste0(dirname(dirname(getwd())),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

In [ ]:
library(forcats)
#library(wesanderson)

### 0 - Read exhaustive analysis

In [ ]:
fe <- fread(paste0(SHARE_DIR, "0_run_fe.csv"))
lr <- fread(paste0(SHARE_DIR, "1_run_lr.csv"))

- Remove redundancies in Fisher exact output
- Some tests are equivalent but signs are flipped

In [ ]:
remove_last <- function(i) i[-length(i)]

In [ ]:
fe_go <- 
fe %>% 
 rw() %>% 
 mu( raw = paste(remove_last(strsplit(feature, "_")[[1]]), collapse = "_")) %>% 
 ug() %>% 
 gb(cohortGo, raw, fisher_pval) %>% 
 mu( rk = row_number(desc(e_nr)), 
     label = factor(ifelse(pr_nr_given_e > pr_nr_overall, "Non-Response Associated", "Response Associated"), 
                   levels = c("Response Associated", "Non-Response Associated"))) %>%
 fi(rk == 1) %>% se(-rk) %>% 
 gb( cohortGo, label ) %>% 
 fi( !grepl("geneset_mp", feature), cohortGo != "Pan-Cancer ## Folinic acid ## Platinum ## Pyrimidine (ant)agonist ## Topoisomerase inhibitor")

### Check the QQ plots

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 6)

In [ ]:
fe_go %>% 
 fi(!grepl("rna_", feature)) %>%
 gb(cohortGo) %>% 
 mu( observed = -log10(sort(fisher_pval)), 
     expected = -log10(ppoints(length(observed))), 
     rk = row_number(max(observed))) %>%
 ar(rk) %>% 
 ug() %>% 
 ggplot(aes(x = expected, y = observed)) + 
 geom_point() + 
 facet_wrap(~fct_reorder(cohortGo, -observed, .fun = min), ncol = 7) + 
 geom_abline(a = 0, b = 1) + 
 go_theme + 
 labs(x = "-Log10( expected p-values )", y = "-Log10( observed p-values )", title = "QQ Plots by Cohort")

#### Data Preparation for Bar plots

In [ ]:
map <- c("e_nr" = "Event + No DCB", "e_r" = "Event + DCB", "ne_nr" = "No Event + No DCB", "ne_r" = "No Event + DCB")

In [ ]:
base <- 
fe_go %>% 
 se( cohortGo, label, feature, fisher_pval, e_nr, e_r, ne_nr, ne_r) %>% 
 ga( event, ct, -cohortGo, -label, -feature, -fisher_pval ) %>% 
 rw() %>% 
 mu(event2 = factor(map[[event]], levels = unname(map))) 

In [ ]:
now <- 
base %>% 
 mu(rna = ifelse(m("rna_", feature), "RNA Feature", "DNA or Clinical Feature" )) %>% 
 mu( feature = ifelse(m("rna_geneset", feature), strsplit(feature, "rna_geneset_")[[1]][2], feature), 
     feature = ifelse(m("gene_set_", feature), strsplit(feature, "gene_set_")[[1]][2], feature),
     feature = str_to_title(feature)) %>% 
 fi(m("Driver", feature)) %>% 
 gb(cohortGo, label, event2, rna) %>% 
 mu(rk = row_number(fisher_pval)) %>% 
 fi(rk < 6) %>% 
 mu(event = !m("No Event", event2), dcb = !m("No DCB", event2))

In [ ]:
alphas <- c("FALSE" = .15, "TRUE" = 1)
response <- c("FALSE" = "#e52f28", "TRUE" = "#7AABD3")

### Send some figures

In [ ]:
cohort_plotter <- function( i = "Skin Melanoma ## Anti-PD-1") {
 now <- now %>% fi(cohortGo == i)
 most <- max(now$ct) + 1.05   

 now %>% 
 #fi(label == "Non-Response Associated") %>% 
 ggplot(aes(x = event2, y = ct, alpha = event, fill = dcb)) + 
 #facet_wrap(~rna + fct_reorder(feature, fisher_pval, .fun = mean), ncol = 5) + 
 facet_wrap(~label + fct_reorder(feature, fisher_pval, .fun = mean), ncol = 5) +    
  geom_bar(stat = "identity") + 
  scale_fill_manual( values = response) +  
  scale_alpha_manual( values = alphas) + 
  go_theme + 
  geom_text(aes(label = ct), vjust = -0.5, color = "black", size = 3) + 
  geom_text(aes(x = 1.6, 
               y = most, 
               label = paste0("p-value: ", formatC(fisher_pval, format = "e", digits = 1))), 
               vjust = -0.5, color = "black", size = 3) + 
  ylim(0,1.1*most) + 
  labs(title = i, y = "Number of Patients\n", x = "Event and Durable Clinical Benefit") + 
  theme(strip.background = element_rect(fill = "white", color = "black"), legend.position = "none")
}

In [ ]:
##table(now$cohortGo)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 7)
cohort_plotter("Pan-Cancer ## Anti-PD-1")

In [ ]:
FIG_DIR <- "/home/josephusset@vhio.org/biomarkers/util/figs/"

In [ ]:
for( i in unique(now$cohortGo)  ) {
 print(i); flush.console()
 ggsave( paste0(FIG_DIR, gsub("/", "_", i), "nr_table_drivers.png"), plot = cohort_plotter(i = i), width = 18, height = 7)
}

### Only non-response events
- Focus on drivers

In [ ]:
now <- 
base %>% 
 mu(rna = ifelse(m("rna_", feature), "RNA Feature", "DNA or Clinical Feature" )) %>% 
 mu( feature = ifelse(m("rna_geneset", feature), strsplit(feature, "rna_geneset_")[[1]][2], feature), 
     feature = ifelse(m("gene_set_", feature), strsplit(feature, "gene_set_")[[1]][2], feature),
     feature = str_to_title(feature)) %>% 
 fi(m("Driver", feature)) %>% 
 gb(cohortGo, label, event2, rna) %>% 
 mu(rk = row_number(fisher_pval)) %>% 
 ug() %>% 
 fi(rk < 1000) %>% 
 mu(event = !m("No Event", event2), dcb = !m("No DCB", event2)) #%>% 
 #fi(fisher_pval < .1)

In [ ]:
index_no_dcb <- now %>% fi(event2 == "Event + DCB", ct == 0) %>% se(cohortGo, label, feature)

In [ ]:
ready <- now %>% ij(index_no_dcb, by = c("cohortGo", "label", "feature"))

In [ ]:
most <- max(ready$ct) + 1.05 

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 10)

In [ ]:
labels = ready %>% gb(cohortGo, label = formatC(fisher_pval, format = "e", digits = 1)) %>% su(y_pos = max(ct)*1.05)

In [ ]:
top <- 
ready %>% 
  fi(feature %in% c("Driver_pathway_immune_evasion_gt50", "Driver_pathway_hippo_gt50", 
                   "Driver_erbb4", "Driver_fbxw7")) %>% 
 #fi(label == "Non-Response Associated") %>% 
 ggplot(aes(x = event2, y = ct, alpha = event, fill = dcb)) + 
 #facet_wrap(~rna + fct_reorder(feature, fisher_pval, .fun = mean), ncol = 5) + 
 facet_wrap(~fct_reorder(cohortGo, fisher_pval, .fun = mean) + feature , ncol = 4, scales = "free") +    
  geom_bar(stat = "identity") + 
  scale_fill_manual( values = response) +  
  scale_alpha_manual( values = alphas) + 
  go_theme + 
  geom_text(aes(label = ct), vjust = -0.5, color = "black", size = 3) + 
  geom_text(aes(x = 1.6, 
               y = 70, 
               label = paste0("p-value: ", formatC(fisher_pval, format = "e", digits = 1))), 
               vjust = -0.5, color = "black", size = 3) + 
  coord_cartesian(ylim = c(0,80)) + 
  labs(title = "Driver Events with No Durable Benefit given Treatment", y = "Number of Patients\n", x = "Event and Durable Clinical Benefit") + 
  theme(strip.background = element_rect(fill = "white", color = "black"), legend.position = "none")

In [ ]:
ready %>% 
  fi(feature %in% c("Driver_pathway_immune_evasion_gt50", "Driver_pathway_hippo_gt50", 
                   "Driver_erbb4", "Driver_fbxw7")) 

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 4)

In [ ]:
top

In [ ]:
ggsave( paste0(FIG_DIR, "Driver_events_with_no_durable_benefit.png"), plot = top, width = 14, height = 10)